In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('C:/Users/Iasonas/PycharmProjects/EntityMatching_Thesis/DataFusion/tutorials')

import fuse
fuse_env = fuse.Fuse(verbose=True)
fuse_session = fuse_env.get_session('test')

# MATCHER='DEDUPE'
MATCHER='AHC'

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\Iasonas\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
INFO:gensim.models.utils_any2vec:loading projection weights from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:gensim.models.ut

In [2]:
if MATCHER=='DEDUPE':
    from matching.dedupe import DedupeMatcher
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                                  'fusion_test', matching_strategy=DedupeMatcher)

if MATCHER=='AHC':
    from matching.ahc_matcher import AgglomerativeHierarchicalClustering
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                               'fusion_test', matching_strategy=AgglomerativeHierarchicalClustering)


In [3]:
fuse_session.train_matchers('../../datasets/current/matched.json')

In [4]:
# x=dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','doctoral_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','Bsc')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Msc','Bsc')
# x=dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('manager','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('coder','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('doctor','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('clinician','doctor')

In [5]:
fuse_session.match_observations()

In [6]:
tr_clusters = fuse_session.find_true_clusters()


INFO:global:Running truth discovery for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Working Experience': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Location': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Name': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Education': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Working Experience': {'pipl.com': 1.493925025312256, 'find.jobs': 1.285198244248522, 'pro-profiles.com': 1.4392167625742331, 'internsteps.com': 1.6094379124341007}, 'Location': {'pipl.com': 1.1143606456362487, 'find.jobs': 1.2452157628599851, 'pro-profiles.com': 1.7117167615545186, 'internsteps.com': 1.7117167615545186}, 'Name': {'pipl.com': 2.517696472610991, 'find.jobs': 0.9267620317414502, 'pro-profiles.com': 2.517696472610991

In [7]:
true_facts = fuse_session.find_true_facts(persist=True, no_weights=False)

INFO:global:Running fusion for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Working Experience': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Location': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Name': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}, 'Education': {'pipl.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Working Experience': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'pro-profiles.com': 4.595119850134589, 'internsteps.com': 4.595119850134589}, 'Location': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'pro-profiles.com': 4.595119850134589, 'internsteps.com': 4.595119850134589}, 'Name': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'pro-profiles.com': 3.8501476017100567, 'internsteps.

In [8]:
true_facts_sort = sorted(true_facts, key=lambda f: (f.ent_id, f.ent_attr), reverse=False)
for f in true_facts_sort:
    print(f.ent_id, f.ent_attr, f.canonicalTuple)

# for f in true_facts:
#     print(f.canonicalTuple)

0 Education {'university': 'MIT', 'year': '1997', 'degree': 'Msc'}
0 Education {'university': 'UC_Berkley', 'year': '1995', 'degree': 'Msc'}
0 Location {'location': 'East P.'}
0 Name {'name': 'Meljssb Johnson'}
0 Working Experience {'company': 'Banks, Reynolds and Goodwin', 'title': 'doctor', 'years': '1992-2006'}
1 Education {'university': 'MIT', 'year': '2017', 'degree': 'Bsc'}
1 Location {'location': 'Tylersiee'}
1 Name {'name': 'Brandy Allen'}
1 Working Experience {'company': 'Hoover-Gonzalez', 'title': 'developer', 'years': '1991-2005'}
2 Education {'university': 'UC_Berkley', 'year': '2003', 'degree': 'Bsc'}
2 Location {'location': 'Matthewsmouth'}
2 Name {'name': 'Reginald Jaackson'}
2 Working Experience {'company': 'Simpson, Mendoza and Carlson', 'title': 'analyst', 'years': '1993-2005'}
3 Education {'university': 'MIT', 'year': '1993', 'degree': 'Msc'}
3 Education {'university': 'UC_Berkley', 'year': '2000', 'degree': 'Bsc'}
3 Location {'location': 'Dicksonburgh'}
3 Name {'nam

In [9]:
eval = fuse_session.evaluate(inferred_facts=true_facts, grd_path='../../datasets/current/eval.json')
print(eval)

Matched = 224.00, Matched facts couples = 224.00, Unmatched true = 5.00, Unmatched inferred = 0.00, Total = 229.00
recall= 0.98
Precision = 1.00
Accuracy = 0.98
f1score = 0.99
{'recall': 0.9781659388646288, 'precision': 1.0, 'f1score': 0.988962472406181}
